In [ ]:
import pandas as pd
import requests
import os
from datetime import datetime

# DraftKings API endpoints
LEAGUE_API = "https://sportsbook-nash.draftkings.com/api/sportscontent/dkusnj/v1/leagues/88808"
EVENT_API = "https://sportsbook-nash.draftkings.com/api/sportscontent/dkusnj/v1/events/{event_id}/categories"
OUTPUT_DIR = "temp_csv"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
    "Accept": "application/json",
}

# Stat types to extract (longest first to avoid partial matches)
STAT_TYPES = [
    "Rushing + Receiving Yards",
    "Passing Touchdowns",
    "Passing Yards",
    "Rushing Yards",
    "Receiving Yards",
    "Receptions",
]

def fetch_upcoming_games():
    """Fetch list of upcoming NFL games."""
    response = requests.get(LEAGUE_API, headers=headers, timeout=15)
    response.raise_for_status()
    data = response.json()
    
    games = []
    for event in data.get("events", []):
        games.append({
            "event_id": event.get("id"),
            "matchup": event.get("name"),
            "start_date": event.get("startEventDate"),
        })
    return games

def fetch_event_odds(event_id):
    """Fetch all odds for a specific event including player props."""
    url = EVENT_API.format(event_id=event_id)
    response = requests.get(url, headers=headers, timeout=15)
    response.raise_for_status()
    return response.json()

def parse_event_odds(data):
    """Parse event odds into DataFrames for game lines and player props."""
    
    event = data.get("events", [{}])[0]
    markets = {m["id"]: m for m in data.get("markets", [])}
    
    # Group selections by market
    market_sels = {}
    for sel in data.get("selections", []):
        mid = sel.get("marketId")
        if mid not in market_sels:
            market_sels[mid] = []
        market_sels[mid].append(sel)
    
    # Parse game lines
    game_lines = {"matchup": event.get("name")}
    
    # Parse player props
    player_props = []
    
    for mid, market in markets.items():
        name = market.get("name", "")
        sels = market_sels.get(mid, [])
        
        # Game lines
        if name == "Moneyline":
            for i, sel in enumerate(sels):
                key = "away_ml" if i == 0 else "home_ml"
                game_lines[key] = sel.get("displayOdds", {}).get("american")
                if i == 0:
                    game_lines["away_team"] = sel.get("label")
                else:
                    game_lines["home_team"] = sel.get("label")
                    
        elif name == "Spread":
            for sel in sels:
                label = sel.get("label")
                if game_lines.get("away_team") and label == game_lines["away_team"]:
                    game_lines["away_spread"] = sel.get("points")
                    game_lines["away_spread_odds"] = sel.get("displayOdds", {}).get("american")
                else:
                    game_lines["home_spread"] = sel.get("points")
                    game_lines["home_spread_odds"] = sel.get("displayOdds", {}).get("american")
                    
        elif name == "Total":
            for sel in sels:
                label = sel.get("label", "")
                if "Over" in label:
                    game_lines["total"] = sel.get("points")
                    game_lines["over_odds"] = sel.get("displayOdds", {}).get("american")
                elif "Under" in label:
                    game_lines["under_odds"] = sel.get("displayOdds", {}).get("american")
        
        # Anytime TD Scorer
        elif name == "Anytime TD Scorer":
            for sel in sels:
                player_props.append({
                    "player": sel.get("label"),
                    "market": "Anytime TD",
                    "line": None,
                    "odds": sel.get("displayOdds", {}).get("american"),
                })
        
        # First TD Scorer
        elif name == "First TD Scorer":
            for sel in sels:
                player_props.append({
                    "player": sel.get("label"),
                    "market": "First TD",
                    "line": None,
                    "odds": sel.get("displayOdds", {}).get("american"),
                })
        
        # Player props (milestone format: "250+" etc)
        else:
            # Check stat types (longest first to avoid partial matches)
            matched_stat = None
            for stat in STAT_TYPES:
                if name.endswith(stat):
                    matched_stat = stat
                    break
            
            if matched_stat:
                player_name = name[:-len(matched_stat)].strip()
                for sel in sels:
                    player_props.append({
                        "player": player_name,
                        "market": matched_stat,
                        "line": sel.get("label"),
                        "odds": sel.get("displayOdds", {}).get("american"),
                    })
    
    return game_lines, pd.DataFrame(player_props)


# Fetch upcoming games
print("Fetching upcoming NFL games...")
games = fetch_upcoming_games()
print(f"Found {len(games)} games\n")

# Get first game
first_game = games[0]
print(f"First game: {first_game['matchup']}")
print(f"Event ID: {first_game['event_id']}\n")

# Fetch full odds for first game
print("Fetching odds and player props...")
event_data = fetch_event_odds(first_game["event_id"])
game_lines, player_props_df = parse_event_odds(event_data)

# Save to CSV
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Game lines
game_lines_df = pd.DataFrame([game_lines])
game_lines_df.to_csv(f"{OUTPUT_DIR}/game_lines.csv", index=False)

# Player props
player_props_df.to_csv(f"{OUTPUT_DIR}/player_props.csv", index=False)

print(f"\n=== GAME LINES ===")
print(game_lines_df.T)

print(f"\n=== PLAYER PROPS ({len(player_props_df)} total) ===")
print(f"Markets: {player_props_df['market'].unique().tolist()}")
print(f"\nSample:")
print(player_props_df.head(20))